In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS1_5classes__HOS.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5
0,2.466699,8.076415,0.130942,0.365797,0
1,2.388250,7.647216,0.118878,0.351855,0
2,2.270058,7.048029,0.147898,0.403758,0
3,2.235512,6.838190,0.157496,0.395403,0
4,2.269395,6.948087,0.165563,0.403978,0


# Data preprocessing

In this step we are going standardize our dataset.

In [4]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

Separete features from labels.

In [5]:
data_std = std_scaler.fit_transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Load classifiers

In [6]:
from sklearn.externals import joblib

random_search_SVM = joblib.load('./Models/classifiers_SVM.pkl')
random_search_MLP = joblib.load('./Models/classifiers_MLP.pkl')

In [7]:
from sklearn.naive_bayes import GaussianNB

bayes_clf = GaussianNB()

In [8]:
svm_clf = random_search_SVM['SVM-RBF'].best_estimator_
mlp_clf = random_search_MLP['MLP'].best_estimator_

In [9]:
classifiers = {'MLP-781': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# Train Classifiers and test them

Stratified cross-validation for model selection are going to be used.

In [10]:
kfolds = 8
clf_outputs = run_classifiers(data_std, data_label, classifiers, kfolds)

# Performance assessment

Saving variables in a dictionary:

In [11]:
results = {}
results['train'] = results_clf(5, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = results_clf(5, clf_outputs['test']['true'], clf_outputs['test']['pred'])

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/HOS/Experiment_03_scikitLearn__allCLF/experiment.py:150: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/HOS/Experiment_03_scikitLearn__allCLF/experiment.py:152: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,3] = (2*metrics_class[i,2]*metrics_c

# Savel results, models and pipeline to a .pkl file

In [12]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('std_scaler', std_scaler),
                               ('ANOVA', selector)
                              ])

NameError: name 'selector' is not defined

In [ ]:
save_models(classifiers)
save_pipeline(data_preprocessing)

# Save results to CSVs and figures

In [ ]:
export_results(results['test'], 'test')
export_results(results['train'], 'train')